In [ ]:
import pandas as pd 
import numpy as np 
from geopy.distance import geodesic 

In [ ]:
streetlights_df = pd.read_csv("/Users/stevenqie/Downloads/Streetlights_with_block.csv")
locations_df = pd.read_csv("/Users/stevenqie/Downloads/Location.csv")
apartments_df = pd.read_csv("/Users/stevenqie/Downloads/Apartments_with_block.csv")
crashes_df = pd.read_csv("/Users/stevenqie/Downloads/Crash_with_block.csv")

In [ ]:
def calculate_distance(lat1, lon1, lat2, lon2):
    return geodesic((lat1, lon1), (lat2, lon2)).meters

proximity_radius = 500 #in meters 

In [ ]:
apartments_df.head()
len(apartments_df)

In [ ]:
streetlights_df.head()

In [ ]:
# create a new column to store the number of streetlights within the proximity radius
apartments_df['num_streetlights'] = 0 
# create a new colum to store the average wattage of those streetlights within the proximity radius
apartments_df['avg_wattage'] = 0

#loop through each apartment 
for index1, apartment in apartments_df.iterrows():
    print(index1)
    #grab its lat and long 
    apartment_location = (apartment["latitude"], apartment["longitude"])
    count = 0 
    wattage_sum = 0 

    #loop through each streetlight and calculate distance betweeen aparmtent and streetlight 
    for index2, streetlight in streetlights_df.iterrows():
        streetlight_location = (streetlight["latitude"], streetlight["longitude"])
        distance = calculate_distance(apartment_location[0], apartment_location[1], streetlight_location[0], streetlight_location[1])
        #check if distance is less than proximity radius
        if distance <= proximity_radius:
            #add to count
            count += 1 
            wattage_sum += streetlight["wattage"] if not np.isnan(streetlight["wattage"]) else 0
    apartments_df.loc[index1, 'num_streetlights'] = count
    apartments_df.loc[index1, 'avg_wattage'] = wattage_sum / count if count > 0 else 0
    print(f"Apartment has {apartments_df.loc[index1, 'num_streetlights']} streetlights within {proximity_radius} meters")
    print(f"Average wattage of streetlights around this apartment is {apartments_df.loc[index1, 'avg_wattage']} watts")



In [ ]:
apartments_df = apartments_df.iloc[:-1]
apartments_df

In [ ]:
crashes_df.head()

In [ ]:
copy = apartments_df.copy()
copy 

In [ ]:
#get crashes per block using group by 
crashes_df = pd.read_csv("/Users/stevenqie/Downloads/Crash_with_block.csv")
crashes_df = crashes_df.groupby("block").size().reset_index(name='num_crashes')
crashes_df = crashes_df[crashes_df['block'] >= 0]
crashes_df = crashes_df.reset_index(drop=True)
crashes_df


In [ ]:
newone = pd.merge(copy, crashes_df, on='block', how='left')
newone = newone.fillna(0)
newone

In [ ]:
#merge with locations to get whether or not an apartment has a police station in it's block 
extranewone = newone.merge(locations_df, on='block', how='left')
extranewone = extranewone.fillna(0)
extranewone

In [ ]:
block_data = extranewone.copy()
block_data.head()

In [ ]:
block_data['streetlight_score'] = (block_data['num_streetlights'] - block_data['num_streetlights'].min()) / (block_data['num_streetlights'].max() - block_data['num_streetlights'].min())

block_data['wattage_score'] = (block_data['avg_wattage'] - block_data['avg_wattage'].min()) / (block_data['avg_wattage'].max() - block_data['avg_wattage'].min())

block_data['crash_score'] = (block_data['num_crashes'].max() - block_data['num_crashes']) / (block_data['num_crashes'].max() - block_data['num_crashes'].min()) 

# Experiment with larger weights for better granularity
block_data['safety_score'] = ((block_data['streetlight_score'] * 60) +   
                              (block_data['wattage_score'] * 40) +       
                              (block_data['has_police_station'] * 40) + 
                              (block_data['crash_score'] * 20))         

# Scale to 0–100 for a wider range
block_data['safety_score'] = (block_data['safety_score'] / block_data['safety_score'].max()) * 100

# Optionally scale down to 0–10
block_data['safety_score'] = (block_data['safety_score'] / 10).round()

In [ ]:
block_data['safety_score'].unique()

In [ ]:
#change the type of safety score to int
block_data['safety_score'] = block_data['safety_score'].astype(int)
block_data.head()
#delete columns safestay_score, num_streetlights_avg_wattage, num_crashes_has_police_station, streetlight_score, wattage_score, crash_score
block_data2 = block_data.drop(columns=['safestay_score', 'num_streetlights', 'avg_wattage', 'num_crashes', 'has_police_station', 'streetlight_score', 'wattage_score', 'crash_score'])

block_data3 = block_data2.rename(columns={"safety_score": "safestay_score"})

#rearrange column ordering 
block_data3 = block_data3[['address', 'safestay_score', 'latitude', 'longitude', 'block']]
block_data3.head()

In [ ]:
block_data3["safestay_score"].value_counts()

In [ ]:
block_data3.head()

In [ ]:
#export to csv 
block_data3.to_csv("/Users/stevenqie/Downloads/final_apartments_with_safestay_score.csv", index=False)
